In [1]:
MAX_NUM_THREADS = 4

import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
#print(mkl_rt.mkl_get_max_threads())
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
def mkl_set_num_threads(cores):
    mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))

mkl_set_num_threads(MAX_NUM_THREADS)
print(f'Number of threads was limited to {mkl_get_max_threads()}.')

Number of threads was limited to 4.


In terminal:
```python
python vbtd_experiment.py --num_threads 4 --likelihood "normal" --guide_type "ppca" --isolate_group "" --orthogonolize_terms "" --normalize 0 --highlight_peak 0 --term_type 'lro' --group_term_type '' --batch_size 80 --data_path "/home/pavel/data/eth80/eth80-cropped-close128" --results_path "./"
```

```python
python vbtd_experiment-smni.py --num_threads 4 --likelihood "normal" --guide_type "ppca" --isolate_group "" --orthogonolize_terms "" --normalize 0 --highlight_peak 0 --term_type 'lro' --group_term_type '' --batch_size 80 --data_path "/home/pavel/data/eth80/eth80-cropped-close128" --results_path "./"
```

In [2]:
import os
import logging

import matplotlib.pyplot as plt
from IPython.display import clear_output

import numpy as np

import pyro
import pyro.optim

import torch

import vbtd_tools
import plot_tools
import vbtd
import learning_tools

from importlib import reload
reload(vbtd_tools);
reload(plot_tools);
reload(vbtd);
reload(learning_tools);

from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import fowlkes_mallows_score

import numpy_tools
import loaders.eth80 as eth80
import loaders.smni_eeg as smni_eeg


In [3]:
data_dirname = '/home/pavel/data'

data_eth80_dirname = os.path.join(data_dirname, 'eth80')
data_smni_dirname = os.path.join(data_dirname, 'eeg_smni')

In [4]:
# load ETH80
data_eth80_path = os.path.join(data_eth80_dirname, 'eth80-cropped-close128')

image_shape = 32
Nclasses, Nobjects = 8, 10
Npixels = int(round(image_shape**2))

data, labels, classes = eth80.eth80_dataset(data_eth80_path, image_shape)
classes_reverse_dict = dict((y, x) for x, y in classes.items())
Nsamples, Nangles, _, _, Ncolors = data.shape
data = numpy_tools.reshape_np(
    data, [Nangles, -1, Ncolors], order='F', use_batch=True
)
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)

n = [Npixels, Nangles, Ncolors]

eth80_dataset = {
    'data': data,
    'n': n,
    'labels': labels,
    'classes': classes,
    'classes_reverse_dict': classes_reverse_dict
}

In [ ]:
# load SMNI EEG
data_smni_path = os.path.join(data_smni_dirname, 'smni_eeg_processed.npz')
df = np.load(data_smni_path)
data, labels = df['data'], df['labels']

Nsubjects, Nchannels, Ntime, Nconditions = data.shape
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)
n = [Ntime, Nchannels, Nconditions]

smni_dataset = {
    'data': data,
    'n': n,
    'labels': labels
}

In [7]:
# config

# likelihood = normal | bernoulli (test feature)
likelihood = 'normal'

# variational distribution = ppca | delta | diag_norm | autodelta
guide_type = 'ppca'

# random seeds
sklearn_random_state = 577
kmeans_random_state = 577

# metrics
ami_average = 'arithmetic'

# model optim config
isolate_group = None # [0, 1, 2]
normalize = False
orthogonolize_terms = None # [0, 1, 2]
highlight_peak = None # 100*Nclasses

# performance scores
usv_scores = {
    'ami': lambda x, y: adjusted_mutual_info_score(
        x, y, average_method=ami_average
    ),
    'ari': adjusted_rand_score,
    'fmi': fowlkes_mallows_score    
}

# plot config
max_rank = 

term_type = 'lro' # cpd | tucker-core | tucker-factor | tt
group_term_type = 'lro'

optim_config = {'lr': 0.005}

In [8]:
tmp = numpy_tools.flatten_np(data)
if likelihood == 'bernoulli':
    tmp = tmp / 255
elif likelihood == 'normal':
    tmp_mean = np.mean(tmp, axis=1, keepdims=True)
    tmp_std = np.std(tmp, axis=1, keepdims=True)
    tmp = (tmp - tmp_mean) / tmp_std
else:
    raise ValueError
    
# def scoring(labels_true, labels_pred):
#     return adjusted_mutual_info_score(labels_true, labels_pred,  average_method=ami_average)
# krs, krs_scores = kmeans_seed_search(X=tmp, y=labels, K=Nclasses, scoring=scoring, random_states=1000)
# np.savez_compressed('ami_kmeans_init_probing', init_ami=krs_scores, sklearn_random_states=krs)
# df = np.load('ami_kmeans_init_probing')
# krs, krs_scores = df['sklearn_random_states'], df['init_ami']
# ind = np.argmax(krs_scores)
# kmeans_random_state = krs[ind]

torch_data = torch.from_numpy(numpy_tools.reshape_np(tmp, n)).to(dtype=torch.float32)  

In [9]:
cp_term_config = {
    'type': 'cp',
    'R': 5
}
lro_term_config = {
    'type': 'lro',
    'L': [3],
    'P': 2
}
tucker_term_config1 = {
    'type': 'tucker',
    'r': [3, 3, 3],
    #'r0': 9
}
tucker_term_config2 = {
    'type': 'tucker',
    'r': [3, 3, 3],
    'r0': 3
}
tt_term_config = {
    'type': 'tt',
    'r': [1]+[3]*len(n),
}

term_config_dict = {
    'cpd': cp_term_config,
    'lro': lro_term_config,
    'tucker-core': tucker_term_config1,
    'tucker-factor': tucker_term_config2,
    'tt': tt_term_config,
    'qtt': None
}

In [10]:
use_gm = True#False

btd_terms_list = [
    {'tensor_config': term_config_dict[term_type], 'bias': False, 'bias_config': cp_term_config},
]*Nclasses

if use_gm:
    gm_config = {
        'tensor_config': term_config_dict[group_term_type], 'bias': False, 'bias_config': cp_term_config,
        #'tensor_config': cp_term_config, 'bias': False, 'bias_config': cp_term_config
    }
else:
    gm_config = None

terms_config = {
    'isotropic': True,
    'btd_terms': btd_terms_list,
    'btd_iterms': None,
}
group_term_config = None if gm_config is None else {
    'isotropic': True,
    'btd_term': gm_config,
    'btd_iterm': None,
}
source_mode = 0


pyro.clear_param_store()
vbtd_model = vbtd.VariationalBTD(
    n,
    likelihood=likelihood,
    terms_config=terms_config,
    group_term_config=group_term_config,
    source_mode=source_mode
)
if guide_type in vbtd_model._auto_guide_types:
    vbtd_model.generate_auto_guide(guide_type)

#torch_data = torch_data.to(device=device, dtype=torch.float)
#vbtd_model = vbtd_model.to(device=device, dtype=torch.float)
#if normalize:
#    vbtd_model.normalize_parameters()

optim = pyro.optim.Adam(optim_config)

log_filename = 'vbtd'
log_filename += f"_terms-isotropic={terms_config['isotropic']:d}"
log_filename += f"_likelihood={likelihood}"
log_filename += f"_guide-type={guide_type}"
log_filename += f"_isolate-group={isolate_group}"
log_filename += f"_normalize={normalize}"
log_filename += f"_orthogonolize-terms={orthogonolize_terms}"
log_filename += f"_highlight-peak={highlight_peak}"
log_filename += f"_term-type={term_type}"

save_filename = f'{log_filename}.npz'

log_filename = f'{log_filename}.log'



results = learning_tools.svi_train_unsupervised_mixture_model(
    torch_data,
    mixture_model=vbtd_model,
    optimizer=optim,
    batch_size=None,
    labels=labels,
    model_field='model_with',
    guide_field='guide_with',
    device='cpu',
    save_results_path=save_filename,
    save_model_path=None,
    best_model_metric_name='ami',
    monitor_norms=False,
    usv_scores=usv_scores,
    display_plots=False,
    max_plate_nesting=1,
    maxitnum=100,
    highlight_peak=highlight_peak,#10*Nclasses
    log_filename=log_filename,
    normalize=normalize,
    isolate_group=isolate_group,
    orthogonolize_terms=orthogonolize_terms,
    expert=True
)

Prior:	 ami=0.084	ari=0.049	fmi=0.178	
Posterior:	 ami=0.020	ari=-0.001	fmi=0.202	
updating factors
SVI loss=4.671e+07


KeyboardInterrupt: 